In [4]:
!pip install scikit-learn



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(ngram_range=(1,3))
v.fit(["Thor Hathodawala is looking for a job"])
v.vocabulary_

{'thor': 12,
 'hathodawala': 2,
 'is': 5,
 'looking': 9,
 'for': 0,
 'job': 8,
 'thor hathodawala': 13,
 'hathodawala is': 3,
 'is looking': 6,
 'looking for': 10,
 'for job': 1,
 'thor hathodawala is': 14,
 'hathodawala is looking': 4,
 'is looking for': 7,
 'looking for job': 11}

In [11]:
corpus = [
    "Thor ate pizza",
    "Loki is tall",
    "Loki is eating pizza"
]

In [22]:
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text)

    filtered_tokens = []

    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return " ".join(filtered_tokens)

preprocess("Thor ate pizza")    

'thor eat pizza'

In [13]:
corpus_processed = [preprocess(text) for text in corpus]
corpus_processed

['thor eat pizza', 'Loki tall', 'Loki eat pizza']

In [14]:
v = CountVectorizer(ngram_range=(1,2))
v.fit(corpus_processed)
v.vocabulary_

{'thor': 7,
 'eat': 0,
 'pizza': 5,
 'thor eat': 8,
 'eat pizza': 1,
 'loki': 2,
 'tall': 6,
 'loki tall': 4,
 'loki eat': 3}

In [16]:
v.transform(["Thor eat pizza"]).toarray()

array([[1, 1, 0, 0, 0, 1, 0, 1, 1]], dtype=int64)

In [17]:
v.transform(["Hulk eat pizza"]).toarray()

array([[1, 1, 0, 0, 0, 1, 0, 0, 0]], dtype=int64)

In [6]:
import pandas as pd

df = pd.read_json("News_Category_Dataset_v3.json", lines=True)
print(df.shape)
df.head()

(209527, 6)


,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [11]:
df.category.value_counts()

category
POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATI

In [7]:
min_samples = 1014
df_politics = df[df.category=="POLITICS"].sample(min_samples, random_state=2022)
df_wellness = df[df.category=="WELLNESS"].sample(min_samples, random_state=2022)
df_entertainment = df[df.category=="ENTERTAINMENT"].sample(min_samples, random_state=2022)
df_travel = df[df.category=="TRAVEL"].sample(min_samples, random_state=2022)

In [8]:
df_balanced = pd.concat([df_politics, df_wellness, df_entertainment, df_travel], axis=0)
df_balanced.category.value_counts()

category
POLITICS         1014
WELLNESS         1014
ENTERTAINMENT    1014
TRAVEL           1014
Name: count, dtype: int64

In [9]:
df_balanced['category_num'] = df_balanced.category.map({
    'POLITICS':0, 'WELLNESS':1, 'ENTERTAINMENT':2, 'TRAVEL':3
})

df_balanced.head()

,link,headline,category,short_description,authors,date,category_num
16014,https://www.huffingtonpost.com/entry/trump-hea...,The Coverage Of Trump’s Big Dumb Body Is Fat W...,POLITICS,"The president, it evidently needs to be said, ...",Travis Waldron,2018-01-18,0
25545,https://www.huffingtonpost.com/entry/dreamers-...,"Dreamers Are People, Not Political Footballs",POLITICS,People should not be reduced to pawns used by ...,"Center for Community Change Action, Contributo...",2017-09-09,0
51291,https://www.huffingtonpost.com/entry/democrats...,Democrats Must Elect Bernie Sanders Senate Min...,POLITICS,The Senate will meet this Wednesday to elect i...,"Linda Milazzo, ContributorParticipatory journa...",2016-11-13,0
41123,https://www.huffingtonpost.com/entry/jcc-lette...,Jewish Leaders Frustrated By Lack Of Progress ...,POLITICS,"In a letter to Attorney General Jeff Sessions,...",Matt Ferner,2017-03-08,0
10450,https://www.huffingtonpost.com/entry/donald-tr...,Donald Trump Roasted For Painfully Awkward Att...,POLITICS,"Well, that didn't seem to go as planned.",Rebecca Shapiro,2018-04-24,0


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_balanced.headline,
    df_balanced.category_num,
    test_size=0.2,
    random_state=2022,
    stratify=df_balanced.category_num
)

In [11]:
print(X_train.shape)
X_train.head()

(3244,)


69836     'SNL' Resurrects Elaine From 'Seinfeld' To Yad...
20146     Ivanka Trump Linked To Fraudster In Trump Pana...
172448            Experience Belgrade Like A Local (PHOTOS)
148640                    4 Scams to Watch out for in Paris
72285     An Open Letter to My Fellow Millennials on Hil...
Name: headline, dtype: object

In [12]:
y_train.value_counts()

category_num
2    811
0    811
3    811
1    811
Name: count, dtype: int64

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

clf = Pipeline([
    ('vectorizer_bow', CountVectorizer(ngram_range=(1,3))),
    ('Multi NB', MultinomialNB())
])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.86      0.82       203
           1       0.79      0.74      0.77       203
           2       0.79      0.71      0.75       203
           3       0.74      0.80      0.77       203

    accuracy                           0.78       812
   macro avg       0.78      0.78      0.78       812
weighted avg       0.78      0.78      0.78       812



In [17]:
y_test[:5]

40481     2
126299    2
154947    3
169934    1
7058      2
Name: category_num, dtype: int64

In [18]:
y_pred[:5]

array([0, 3, 3, 3, 2], dtype=int64)

In [23]:
df_balanced['preprocessed_txt'] = df_balanced.headline.apply(preprocess)

In [24]:
df_balanced.head()

,link,headline,category,short_description,authors,date,category_num,preprocessed_txt
16014,https://www.huffingtonpost.com/entry/trump-hea...,The Coverage Of Trump’s Big Dumb Body Is Fat W...,POLITICS,"The president, it evidently needs to be said, ...",Travis Waldron,2018-01-18,0,Coverage Trump big Dumb body Fat stupidity
25545,https://www.huffingtonpost.com/entry/dreamers-...,"Dreamers Are People, Not Political Footballs",POLITICS,People should not be reduced to pawns used by ...,"Center for Community Change Action, Contributo...",2017-09-09,0,dreamer People political football
51291,https://www.huffingtonpost.com/entry/democrats...,Democrats Must Elect Bernie Sanders Senate Min...,POLITICS,The Senate will meet this Wednesday to elect i...,"Linda Milazzo, ContributorParticipatory journa...",2016-11-13,0,Democrats Elect Bernie Sanders Senate Minority...
41123,https://www.huffingtonpost.com/entry/jcc-lette...,Jewish Leaders Frustrated By Lack Of Progress ...,POLITICS,"In a letter to Attorney General Jeff Sessions,...",Matt Ferner,2017-03-08,0,jewish leader Frustrated Lack Progress Bomb Th...
10450,https://www.huffingtonpost.com/entry/donald-tr...,Donald Trump Roasted For Painfully Awkward Att...,POLITICS,"Well, that didn't seem to go as planned.",Rebecca Shapiro,2018-04-24,0,Donald Trump Roasted Painfully Awkward Attempt...


In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    df_balanced.preprocessed_txt,
    df_balanced.category_num,
    test_size=0.2,
    random_state=2022,
    stratify=df_balanced.category_num
)

In [26]:
clf = Pipeline([
    ('vectorizer_bow', CountVectorizer(ngram_range=(1,3))),
    ('Multi NB', MultinomialNB())
])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.88      0.83       203
           1       0.79      0.71      0.75       203
           2       0.81      0.74      0.77       203
           3       0.78      0.83      0.80       203

    accuracy                           0.79       812
   macro avg       0.79      0.79      0.79       812
weighted avg       0.79      0.79      0.79       812

